In [9]:
import pandas as pd

In [10]:
def read_protein_file(file_path):
    with open(file_path, "r") as file:
        # Read the entire contents of the file
        seq = ""
        str = ""

        sequences = []
        strings = []
        count = 0
        for line in file:
            if count < 8:
                pass
            count += 1

            line = line.strip()
            if line == "<>":
                seq = ""
                str = ""
            elif line == "<end>" or line=="end":
                sequences.append(seq)
                strings.append(str)
            else:
                letters = line.split(" ")
                if len(letters) == 2:
                    seq += letters[0]
                    str += letters[1] if letters[1] != "_" else "c"

        print(sequences)
        return sequences, strings


In [11]:
train_seq, train_str = read_protein_file("data/protein-secondary-structure.train")
test_seq, test_str = read_protein_file("data/protein-secondary-structure.test")

['GVGTVPMTDYGNDVEYYGQVTIGTPGKSFNLNFDTGSSNLWVGSVQCQASGCKGGRDKFNPSDGSTFKATGYDASIGYGDGSASGVLGYDTVQVGGIDVTGGPQIQLAQRLGGGGFPGDNDGLLGLGFDTLSITPQSSTNAFDQVSAQGKVIQPVFVVYLAASNISDGDFTMPGWIDNKYGGTLLNTNIDAGEGYWALNVTGATADSTYLGAIFQAILDTGTSLLILPDEAAVGNLVGFAGAQDAALGGFVIACTSAGFKSIPWSIYSAIFEIITALGNAEDDSGCTSGIGASSLGEAILGDQFLKQQYVVFDRDNGIRLAPVA', 'AQCEATIESNDAMQYDLKEMVVDKSCKQFTVHLKHVGKMAKSVMGHNWVLTKEADKEGVATDGMNAGLAQDYVKAGDTRVIAHTKVIGGGESDSVTFDVSKLTPGEAYAYFCSFPGHWAMMKGTLKLSN', 'SVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPGHSALMKGTLTLK', 'ALWQFNGMIKCKIPSSEPLLDFNNYGCYCGLGGSGTPVDDLDRCCQTHDNCYKQAKKLDSCKVLVDNPYTNNYSYSCSNNEITCSSENNACEAFICNCDRNAAICFSKVPYNKEHKNLDKKNC', 'HWGYGKHDGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNDGHAFNVEFDDSEDKAVLKGGPLDGTYRLIQFHFHWGSLDGEGSQHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFDGEGEPEELMVDNWRPAQPLKNRQIKASF', 'GGGARSGDDVVAKYCNACHGT

Get all the test lengths

In [12]:
protein_lengths = []
for i in test_seq:
    protein_lengths.append(len(i))
protein_lengths

[306, 108, 113, 322, 62, 212, 281, 218, 198, 107, 461, 149, 220, 334, 35]

Create sliding windows

Note. we need to split the sequences up, but since our test input is of different lengths, what do we do then?

When your test data contains protein sequences of different lengths compared to your training data, you need to handle this discrepancy appropriately. Here are a few strategies you can consider:

Padding:

Pad shorter sequences in the test data to match the length of the longest sequence in your training data.
You can pad the sequences with a special token (e.g., all zeros) so that the neural network can recognize them as padding.
During inference, you would need to trim the predictions to the original length of the test sequence.
Dynamic Input Shape:

Modify your neural network architecture to accept variable-length input sequences.
Use techniques like masking to handle variable-length sequences effectively.
Many deep learning frameworks, such as TensorFlow and PyTorch, support dynamic input shapes and masking.
Bucketing or Binning:

Group sequences of similar lengths together and pad each group separately to the maximum length within that group.
This approach reduces the amount of padding required, leading to more efficient training.
You can organize your test data into buckets or bins based on sequence lengths and process each bucket separately during evaluation.
Batch Processing:

While processing batches during testing, group sequences of similar lengths together.
This allows you to minimize padding within each batch, improving computational efficiency.

One hot encoding the sequence

Encoding Amino Acids:

First, you need to encode each amino acid into a numerical representation.
Common encoding methods include one-hot encoding and embedding.
In one-hot encoding, each amino acid is represented as a binary vector where only one element is 1 (indicating the presence of that amino acid).
In embedding, each amino acid is mapped to a low-dimensional vector space.
Padding:

After encoding amino acids, you can proceed with padding sequences as discussed earlier.
Pad the sequences to match the length of the longest sequence in your training data.

PERFORM NEURAL NETWORK??? use RNN or CNN (keras, tensorflow, pytorch)

https://www.pnas.org/doi/epdf/10.1073/pnas.86.1.152

sliding window = 17

In [13]:
def split_based_on_windows(data_seq, data_str, W=17):
    all_sequences = []
    all_strings = []
    print([len(i) for i in data_seq])
    for i in range(len(data_seq)):
        sequences = [data_seq[i][j:j+W] for j in range(0, len(data_seq[i]), W)]
        strings = [data_str[i][j:j+W] for j in range(0, len(data_str[i]), W)]
        # PADDING
        if len(sequences[-1]) != W:
            sequences[-1] = sequences[-1] + "X"*((W-len(sequences[-1])))
        if len(strings[-1]) != W:
            strings[-1] = strings[-1] + "X"*((W-len(strings[-1])))

        all_sequences += sequences
        all_strings += strings

    train_df = pd.DataFrame({"sequence": all_sequences, "string": all_strings})
    return train_df


In [14]:
train_df = split_based_on_windows(train_seq, train_str)
test_df = split_based_on_windows(test_seq, test_str)

[324, 129, 125, 123, 256, 83, 111, 108, 46, 71, 118, 103, 136, 240, 207, 146, 54, 147, 29, 174, 70, 67, 149, 145, 85, 239, 30, 329, 130, 164, 129, 153, 153, 26, 124, 111, 36, 107, 293, 124, 65, 56, 247, 194, 318, 323, 85, 256, 127, 293, 146, 106, 95, 87, 75, 57, 153, 222, 325, 61, 114, 114, 181, 141, 151, 107, 184, 478, 207, 112, 237, 98, 146, 99, 415, 230, 224, 50, 316, 82, 437, 159, 138, 222, 153, 307, 333, 58, 54, 374, 498]
[306, 108, 113, 322, 62, 212, 281, 218, 198, 107, 461, 149, 220, 334, 35]


In [15]:
train_df.shape

(934, 2)

In [16]:
test_df.shape

(190, 2)

In [17]:
test_df.tail()

,sequence,string
185,VKLVSWYDNEFGYSERV,eeeecccchhhhhhhhh
186,VDLMAHMASKEXXXXXX,hhhhhhhhhhcXXXXXX
187,SIPPEVKFNKPFVFLMI,cccceeecccceeeeee
188,EQNTKSPLFMGKVVNPT,ecccceeeeeeeecccc
189,QXXXXXXXXXXXXXXXX,cXXXXXXXXXXXXXXXX
